<a href="https://colab.research.google.com/github/Aminaa88/Langchain_RAG_Project/blob/main/project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

from pinecone import Pinecone,ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
index_name = "online-rag-project2"

try:
    pc.delete_index(index_name)
    print(f"Index '{index_name}' deleted successfully.")
except Exception as e:
    print(f"Error deleting index: {e}")

# Now you can recreate the index
pc.create_index(
   name=index_name,
   dimension=768,
   metric="cosine",
   spec=ServerlessSpec(cloud="aws" , region="us-east-1"),
)
index = pc.Index(index_name)

Index 'online-rag-project2' deleted successfully.


In [ ]:
import time

index_name = "online-rag-project" # Choose a new name

pc.create_index(
   name=index_name,
   dimension=768,
   metric="cosine",
  spec=ServerlessSpec(cloud="aws" , region="us-east-1"),
)
index = pc.Index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'a992268acb0778be74162036111b254a', 'Date': 'Sun, 05 Jan 2025 21:00:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index,embedding=embeddings)

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="This is my document.",
    metadata={"source": "tweet"},
)

In [ ]:
document_1

Document(metadata={'source': 'tweet'}, page_content='This is my document.')

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="Islamabad is the capital of pakistan.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather is good.",
    metadata={"source": "tweet"},
)

document_3 = Document(
    page_content="Mango is amazing fruit.",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
]

uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)




['c9a32a31-05eb-438b-9775-9520472b9828',
 '6ed2ec8e-dcdb-4508-92cf-93d11dec117c',
 'aeb6ae7e-b3d6-4b6d-bc6e-1cf0d2dca0db']

In [ ]:
len(documents)

3

In [ ]:
from uuid import uuid4
uuid4()

UUID('c718edf0-4569-449f-b4a7-228a18142999')

In [ ]:
results = vector_store.similarity_search(
    "langchain provide abstraction to make working with llm easy?",
     k=3
)
for res in results:
  print(f"*{res.page_content} [{res.metadata}]")


*Mango is amazing fruit. [{'source': 'tweet'}]
*The weather is good. [{'source': 'tweet'}]
*Islamabad is the capital of pakistan. [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "How's the weather?",
)
for res, score in results:
  print(f"*[SIM={score:3f}] {res.page_content} [{res.metadata}]")


*[SIM=0.676678] The weather is good. [{'source': 'tweet'}]
*[SIM=0.529149] Mango is amazing fruit. [{'source': 'tweet'}]
*[SIM=0.524054] Islamabad is the capital of pakistan. [{'source': 'tweet'}]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [ ]:
def answer_to_user(query: str):
  # for vector searching
  vector_results = vector_store.similarity_search_with_score(query)
  #vector results+user querry pass to  model
  final_answer = llm.invoke(f"ANSWER THIS USER QUERY: {query}, Here are some refrences to anwer{vector_results}")
  ChatGoogleGenerativeAI(results, query)
  return final_answer

In [ ]:
answer_to_user("Langchain provides abstraction to make working with llms easy")

AIMessage(content='Langchain simplifies working with LLMs.  Based on the provided references (tweets and their associated similarity scores), we have the following information:\n\n* **Tweet 1:** "Mango is amazing fruit." (Similarity score: 0.466)\n* **Tweet 2:** "The weather is good." (Similarity score: 0.426)\n* **Tweet 3:** "Islamabad is the capital of Pakistan." (Similarity score: 0.423)\n\nThe similarity scores suggest a relationship between these tweets and some unspecified query (the query is not provided).  Without knowing the original query, we cannot definitively say what the relevance of these tweets is.  However, the scores indicate that the first tweet ("Mango is amazing fruit.") has the strongest relationship to the unknown query.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-565c3f52-40fd-4d51-8b91-89239aa4df9a-0', usage_metadata={'input_tokens': 231, 'outp

In [ ]:
answer = answer_to_user("Langchain provides abstraction to make working with llms easy")

In [ ]:
answer.content

"Langchain simplifies working with LLMs.  Based on the provided references, which are documents and their associated similarity scores, we can infer that the documents contain various pieces of information:\n\n* **Mango is an amazing fruit.** (High similarity score, suggesting it might be relevant to a query)\n* **The weather is good.** (Medium similarity score)\n* **Islamabad is the capital of Pakistan.** (Medium similarity score)\n\nWithout knowing the original user query, it's impossible to definitively answer how these documents are relevant.  However, Langchain likely retrieved these documents because they were judged to be somewhat related to the user's input.  The similarity scores indicate the degree of relevance Langchain assigned to each document.  To provide a more specific answer, please provide the original user query.\n"